In [68]:
doughLength=500

import pandas as pd
dataset = {'length':[4,8,2,5],'value':[6,12,1,8],'maximum allowed defects':[{'a': 4, 'b': 2, 'c': 3},{'a': 5, 'b': 4, 'c': 4},{'a': 1, 'b': 2, 'c': 1},{'a': 2, 'b': 3, 'c': 2}]}

df = pd.DataFrame(dataset)
display(df)

defects = pd.read_csv('defects.csv')
defects.sort_values(by=['x'],inplace=True)
display(defects)

class Biscuit : 
    def __init__(self, length, value, maximum_allowed_defects) :
        self.length = length
        self.value = value
        self.maximum_allowed_defects = maximum_allowed_defects

    def __str__(self) :
        if self.value==0 :
            return f"empty space"
        return f"biscuit of length {self.length} and value {self.value}"

    def empty_space() :
        return Biscuit(1,0,{'a':0,'b':0,'c':0})


class dought : 
    def __init__(self, biscuits) :
        self.biscuits = biscuits
    
    def next_x(self) :
        return sum([b.length for b in self.biscuits])
    
    def number_of_defects(self,x,length) :
        interval=defects[(defects['x']>=x) & (defects['x']<x+length)]
        return {'a':interval[interval['class']=='a'].count().astype(int)[0],
                'b':interval[interval['class']=='b'].count().astype(int)[0],
                'c':interval[interval['class']=='c'].count().astype(int)[0]}

    def canbeplaced(self, biscuit) :
        if biscuit.length <= doughLength-self.next_x() :
            number_of_defects=self.number_of_defects(self.next_x(),biscuit.length)
            if (number_of_defects['a'] <= biscuit.maximum_allowed_defects['a'] and number_of_defects['b'] <= biscuit.maximum_allowed_defects['b'] and number_of_defects['c'] <= biscuit.maximum_allowed_defects['c']) :
                return True
        return False
    
    def Value(self) :
        return sum([b.value for b in self.biscuits])

    def leftemptyspace(self) :
        self.biscuits.append(Biscuit.empty_space())

    def place(self, biscuit) :
        if self.canbeplaced(biscuit) :
            self.biscuits.append(biscuit)
            return True
        return False

    def printDough(self) :
        x=0
        for b in self.biscuits :
            print(f"{b} on the interval [{x},{x+b.length}[")
            x+=b.length
        print(f"total value : {self.Value()}, number of biscuits : {len(self.biscuits)-len([b for b in self.biscuits if b.value==0])}")

Biscuit0 = Biscuit(4,6,{'a':4,'b':2,'c':3})
Biscuit1 = Biscuit(8,12,{'a':5,'b':4,'c':4})
Biscuit2 = Biscuit(2,1,{'a':1,'b':2,'c':1})
Biscuit3 = Biscuit(5,8,{'a':2,'b':3,'c':2})




,length,value,maximum allowed defects
0,4,6,"{'a': 4, 'b': 2, 'c': 3}"
1,8,12,"{'a': 5, 'b': 4, 'c': 4}"
2,2,1,"{'a': 1, 'b': 2, 'c': 1}"
3,5,8,"{'a': 2, 'b': 3, 'c': 2}"


,x,class
479,0.700561,a
141,0.848185,a
142,1.769825,a
493,2.207230,b
125,2.313038,a
...,...,...
219,496.411441,a
449,496.429679,b
243,497.370667,a
246,498.455612,c


In [ ]:
doughLength=500

import pandas as pd
dataset = {'length':[4,8,2,5],'value':[6,12,1,8],'maximum allowed defects':[{'a': 4, 'b': 2, 'c': 3},{'a': 5, 'b': 4, 'c': 4},{'a': 1, 'b': 2, 'c': 1},{'a': 2, 'b': 3, 'c': 2}]}

df = pd.DataFrame(dataset)
display(df)

defects = pd.read_csv('defects.csv')
defects.sort_values(by=['x'],inplace=True)
display(defects)

class Biscuit : 
    def __init__(self, length, value, maximum_allowed_defects) :
        self.length = length
        self.value = value
        self.maximum_allowed_defects = maximum_allowed_defects

    def __str__(self) :
        if self.value==0 :
            return f"empty space"
        return f"biscuit of length {self.length} and value {self.value}"

    def empty_space() :
        return Biscuit(1,0,{'a':0,'b':0,'c':0})


class dought : 
    def __init__(self, biscuits) :
        self.biscuits = biscuits
    
    def next_x(self) :
        return sum([b.length for b in self.biscuits])
    
    def number_of_defects(self,x,length) :
        interval=defects[(defects['x']>=x) & (defects['x']<x+length)]
        return {'a':interval[interval['class']=='a'].count().astype(int)[0],
                'b':interval[interval['class']=='b'].count().astype(int)[0],
                'c':interval[interval['class']=='c'].count().astype(int)[0]}

    def canbeplaced(self, biscuit) :
        if biscuit.length <= doughLength-self.next_x() :
            number_of_defects=self.number_of_defects(self.next_x(),biscuit.length)
            if (number_of_defects['a'] <= biscuit.maximum_allowed_defects['a'] and number_of_defects['b'] <= biscuit.maximum_allowed_defects['b'] and number_of_defects['c'] <= biscuit.maximum_allowed_defects['c']) :
                return True
        return False
    
    def Value(self) :
        return sum([b.value for b in self.biscuits])

    def leftemptyspace(self) :
        self.biscuits.append(Biscuit.empty_space())

    def place(self, biscuit) :
        if self.canbeplaced(biscuit) :
            self.biscuits.append(biscuit)
            return True
        return False

    def printDough(self) :
        x=0
        for b in self.biscuits :
            print(f"{b} on the interval [{x},{x+b.length}[")
            x+=b.length
        print(f"total value : {self.Value()}, number of biscuits : {len(self.biscuits)-len([b for b in self.biscuits if b.value==0])}")

Biscuit0 = Biscuit(4,6,{'a':4,'b':2,'c':3})
Biscuit1 = Biscuit(8,12,{'a':5,'b':4,'c':4})
Biscuit2 = Biscuit(2,1,{'a':1,'b':2,'c':1})
Biscuit3 = Biscuit(5,8,{'a':2,'b':3,'c':2})




,length,value,maximum allowed defects
0,4,6,"{'a': 4, 'b': 2, 'c': 3}"
1,8,12,"{'a': 5, 'b': 4, 'c': 4}"
2,2,1,"{'a': 1, 'b': 2, 'c': 1}"
3,5,8,"{'a': 2, 'b': 3, 'c': 2}"


,x,class
479,0.700561,a
141,0.848185,a
142,1.769825,a
493,2.207230,b
125,2.313038,a
...,...,...
219,496.411441,a
449,496.429679,b
243,497.370667,a
246,498.455612,c


# Informed search : Greedy algorithm implementation

This Algoritm tries to place the highest possible value by unit biscuit. If it can't place any biscuit, it places an empty space.

In [69]:
roll = dought([])
biscuits = [Biscuit0,Biscuit1,Biscuit2,Biscuit3]
#sort biscuits by value/length ratio (score/unit)
biscuits.sort(key=lambda x: x.value/x.length, reverse=True)

while roll.next_x()<doughLength:
    i=0
    placed=False
    #try to place the biscuit with the highest score/unit
    while i<len(biscuits) and placed==False:
        placed=roll.place(biscuits[i])
        i+=1
    #if no biscuit was placed, add an empty space
    if i==len(biscuits) and placed==False:
        roll.leftemptyspace()
roll.printDough()

biscuit of length 8 and value 12 on the interval [0,8[
biscuit of length 8 and value 12 on the interval [8,16[
biscuit of length 5 and value 8 on the interval [16,21[
biscuit of length 4 and value 6 on the interval [21,25[
biscuit of length 4 and value 6 on the interval [25,29[
biscuit of length 5 and value 8 on the interval [29,34[
biscuit of length 2 and value 1 on the interval [34,36[
biscuit of length 4 and value 6 on the interval [36,40[
biscuit of length 4 and value 6 on the interval [40,44[
biscuit of length 4 and value 6 on the interval [44,48[
biscuit of length 2 and value 1 on the interval [48,50[
biscuit of length 4 and value 6 on the interval [50,54[
biscuit of length 5 and value 8 on the interval [54,59[
biscuit of length 5 and value 8 on the interval [59,64[
biscuit of length 4 and value 6 on the interval [64,68[
biscuit of length 5 and value 8 on the interval [68,73[
biscuit of length 4 and value 6 on the interval [73,77[
biscuit of length 5 and value 8 on the interval [

## Constraint satisfaction algorithm :

In [ ]:
from ortools.sat.python import cp_model

def optimize_biscuit_production(df, doughLength, defects):
    model = cp_model.CpModel()

    # Create a variable for each biscuit type (count of each type)
    biscuit_vars = [model.NewIntVar(0, doughLength // df['length'][i], f'biscuit_{i}') for i in range(len(df))]

    # Constraint: Total length of biscuits should not exceed dough length
    model.Add(sum(df['length'][i] * biscuit_vars[i] for i in range(len(df))) <= doughLength)

    # Add other constraints like defects here...

    # Objective: Maximize the total value
    total_value = sum(df['value'][i] * biscuit_vars[i] for i in range(len(df)))
    model.Maximize(total_value)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print("Total Value:", solver.ObjectiveValue())
        for i in range(len(df)):
            print(f'Biscuit {i}:', solver.Value(biscuit_vars[i]))
    else:
        print("No optimal solution found.")

# Example usage
optimize_biscuit_production(df, doughLength, defects)
